In [ ]:
import pandas as pd

In [ ]:
data1 = pd.read_csv('data/2017firsthalf.txt', sep="\t", encoding='latin-1', dtype={'componentText': str})
data2 = pd.read_csv('data/2017secondhalf.txt', sep="\t", encoding='latin-1', dtype={'componentText': str})
data3 = pd.read_csv('data/2018.txt', sep="\t", encoding='latin-1', dtype={'componentText': str})

In [ ]:
transcript_train = pd.concat([data1, data2, data3])
#transcript_train = data1

In [ ]:
transcript_train.describe()

In [ ]:
transcript_train.info()

In [ ]:
transcript_train.head()

In [ ]:
transcript_train['transcriptcreationdateUTC'] = pd.to_datetime(transcript_train['transcriptcreationdateUTC'])

In [ ]:
transcript_train.info()

In [ ]:
import spacy
from spacy import displacy

In [1]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz

     |████████████████████████████████| 11.1MB 1.9MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-cp37-none-any.whl size=11074435 sha256=492e6f076e182586a291165d67da764dab851cf28397bc7c568034796b994920
  Stored in directory: /Users/junli/Library/Caches/pip/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import en_core_web_sm

In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
#transcript_train['componentText'] = transcript_train['componentText'].to_string()

In [ ]:
%%time
transcript_train['parsed_componentText'] = transcript_train.iloc[:,2].apply(lambda x: nlp(x))
#test = nlp(transcript_train.iloc[:,2])

In [ ]:
transcript_train.iloc[:,3][0]

In [ ]:
# lemmatization and removing stop words
transcript_train['parsed_componentText_lemma'] = transcript_train.iloc[:,3].apply(lambda text: 
                                          " ".join(token.lemma_ for token in text if not token.is_stop))

In [ ]:
transcript_train.iloc[:,4][0]

In [ ]:
# Adding customized stop words
# # New stop words list 
# customize_stop_words = [
#     'attach'
# ]

# # Mark them as stop words
# for w in customize_stop_words:
#     nlp.vocab[w].is_stop = True

In [ ]:
transcript_train['month'] = transcript_train['transcriptcreationdateUTC'].apply(lambda x: x.month)

In [ ]:
transcript_train1=transcript_train.groupby(['objectId','month'])['parsed_componentText_lemma'].apply(' '.join).reset_index()

In [ ]:
transcript_train1

In [ ]:
# test = list(transcript_train1.iloc[:1,2].apply(lambda x: x.noun_chunks))
# Modeling bigrams and trigrams

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Gensim’s simple_preprocess() is great for this. Additionally I have set deacc=True to remove the punctuations.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#data_words = list(sent_to_words(data))

In [ ]:
transcript_train1['list_of_words'] = list(sent_to_words(transcript_train1['parsed_componentText_lemma']))

In [ ]:
transcript_train1.head()

In [ ]:
# Build the bigram and trigram models
transcript_train1['bigram'] = gensim.models.Phrases(transcript_train1['list_of_words'], min_count=5, threshold=10) # higher threshold fewer phrases.

# trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

In [ ]:
%%time
# # Faster way to get a sentence clubbed as a trigram/bigram
transcript_train1['bigram_mod'] = transcript_train1['bigram'].apply(lambda x: gensim.models.phrases.Phraser(x))
# trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
transcript_train1['bigram_text'] = transcript_train1['bigram_mod'].apply(lambda x: x[transcript_train1['list_of_words']])

In [ ]:
# Create Dictionary
# transcript_train1['id2word'] = transcript_train1['bigram_text'].apply(lambda x: corpora.Dictionary(x))


In [ ]:
%%time
id2word = corpora.Dictionary(transcript_train1['bigram_text'][0])

In [ ]:
corpus = [id2word.doc2bow(x) for x in transcript_train1['bigram_text'][0]]

In [ ]:
texts = []
for i in range(len(transcript_train1['bigram_text'][0])):
    texts.append(transcript_train1['bigram_text'][0][i])
    
    

In [ ]:
%%time
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=25,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

So how to infer pyLDAvis’s output?

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

Alright, if you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.

We have successfully built a good looking topic model.

Given our prior knowledge of the number of natural topics in the document, finding the best model was fairly straightforward.

Upnext, we will improve upon this model by using Mallet’s version of LDA algorithm and then we will focus on how to arrive at the optimal number of topics given any large corpus of text.

In [ ]:
# How to find the optimal number of topics for LDA?
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
%%time
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=2, limit=40, step=2)

In [ ]:
# Show graph
limit=40; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Emotion and Sentiment Analysis

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
transcript_train1['vader'] = transcript_train1.iloc[:,2].apply(lambda x: sid.polarity_scores(x))

In [ ]:
transcript_train1['vader'][2]

In [ ]:
neg_score = list(transcript_train1['vader'].apply(lambda x: x['neg']))

In [ ]:
plt.hist(neg_score)

In [ ]:
pos_score = list(transcript_train1['vader'].apply(lambda x: x['pos']))
plt.hist(pos_score)

In [ ]:
#random cut forest to filter out outlier
# PCA
# classification/regression

#visualization
# word cloud, feature importance, S&P global leaflet, 